In [0]:
import pandas as pd

FILE = "committee_utterances.tsv"
path = "drive/My Drive/Colab Notebooks/466-proj2/"

df = pd.read_csv(path + FILE, sep='\t')

##Prepare dataset
Get a subset of the data to include records from only the top N speakers


In [0]:
# Combine First and last name
names = []
for index,row in df.iterrows():
    speaker = row['last'] + " " + row['first']
    names.append(speaker)

# add to dataframe
df["full_name"] = names

In [0]:
# top 100 speakers with the most records in the dataframe
top_100_speakers = df.pivot_table(index=['full_name'], aggfunc='size')
top_100_speakers = df.groupby(['full_name'])['full_name']\
                .count()\
                .reset_index(name='count')\
                .sort_values(['count'], ascending=False)\
                .head(100)

In [0]:
df_top_100 = df[df.full_name.isin(top_100_speakers['full_name'])]

In [333]:
N = 50

speaker_utter_cnt = []
for name in top_100_speakers.full_name:
    temp_df = df_top_100[df_top_100.full_name == name]

    tot_words = 0
    for t in temp_df.text:
        tot_words += len(t.split())

    speaker_utter_cnt.append((name, tot_words))
speaker_utter_cnt.sort(key=lambda tup: tup[1], reverse=True)    

top_N_speakers = [tup[0] for tup in speaker_utter_cnt[:N]]
df_top_N = df_top_100[df_top_100.full_name.isin(top_N_speakers)]

print("Total speakers: ", N)
print("(First, the top 100 speakers were chosen based on number of records in the dataset.")
print("Then, we narrowed down the selection to the top ", N, " speakers based on number of utterances.)")
print()

Total speakers:  50
(First, the top 100 speakers were chosen based on number of records in the dataset.
Then, we narrowed down the selection to the top  50  speakers based on number of utterances.)



##Features

In [334]:

import nltk, random, spacy
from nltk.corpus import stopwords

nltk.download('stopwords')    #these are only for Collab, on Frank the "download"is not necessary
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets') 
nltk.download('punkt')

stemmer = nltk.stem.porter.PorterStemmer() #NLTK's built-in stemmer resource
nlp = spacy.load("en_core_web_sm")
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#uses NLTK's built in word tokenizer. Output is a list now.
def myTokenizer(text):
  return nltk.word_tokenize(text)

#POS tagger.
#input is a list of words, output is a list of tuples (Word, Tag)
##Do you want to know what the tags mean? Run this: nltk.help.upenn_tagset()
def getPOS(tokenized):
  return nltk.pos_tag(tokenized)

#This removes all the stop words, and actually also punctuation and non-alphabetic words, and makes all lower case
#you can edit your own version
def filterTokens(tokenized):
  return [token.lower() for token in tokenized if token.isalpha() and token.lower() not in stopwords.words('english')]

#Using the NLTK stemmer
def stemming(tokenized):
  return [stemmer.stem(token) for token in tokenized]

def lemmatize(tokenized):
    doc = nlp(' '.join(tokenized))
    return [token.lemma_ for token in doc]

# text is lemmatized, filtered, and tokenized
def mostCommon(texts, N):
    vocab = []
    for text in texts:
        tokenized = lemmatize(filterTokens(myTokenizer(text)))
        vocab.extend(tokenized)
    
    freqdist = nltk.FreqDist(vocab) 
    freqdist = freqdist.most_common()
    return freqdist[:N]

def processWords(tokenized):
    vocab = {}
    for token in tokenized:
        vocab[token] = True
    return vocab

def processSents(text):  # return [(sent, word_count)]
    sents = sent_tokenizer.tokenize(text)
    return [(sent, len(sent.split())) for sent in sents]

def getEntities(text):
    doc = nlp(text)
    
    summary = {}
    entities = {}
    # orgs = []
    # persons = []
    # gpes = []
    # other = []
    for ent in doc.ents:
        # 1 build summary of all entities
        if ent in summary:
            summary["ent_cnt(" + ent.label_ + ")"] += 1
        else:
            summary["ent_cnt(" + ent.label_ + ")"] = 1
            
        entities["contains_entity(" + ent.text +")"] = True
        # # 2 get list of orgs mentioned
        # if (ent.label_ == 'ORG'):
        #     orgs.append(ent.text)
        # # 3 get list of people mentioned
        # elif (ent.label_ == 'PERSON'):
        #     persons.append(ent.text)
        # elif (ent.label_ == 'GPE'):
        #     gpes.append(ent.text)            
        # else:
        #     other.append(ent.text)
    # return summary, orgs, persons, gpes, other
    return summary, entities

    
# Extract features of a single text
def processText(text):
    features = {}

    cleaned = lemmatize(filterTokens(myTokenizer(text)))        

    if (PREPROCESS_STAGE):
        svs = speakerVocabScore(cleaned, num_of_speakers, TOP_VOCAB)
        features.update(svs)

    # word features
    words = processWords(cleaned)
    features.update(words)

    # sentence features
    sent_lengths = [sent[1] for sent in processSents(text)]
    avg_sent = sum(sent_lengths) / len(sent_lengths)
    features['avg_sent'] = avg_sent

    # entity features
    summary, entities = getEntities(text)
    features.update(summary)
    features.update(entities)
    
    return features


# -------------------------------------------------------------------------
# Extract additional features using the training set
# - this takes about 2 minutes to do
# -------------------------------------------------------------------------
# given a speaker and dataset, returns a list of corresponding texts
def getSpeakerTexts(speaker, data):
    print('Speaker ', speaker, ' - total texts: ', len(data[data.full_name == speaker]['text']))
    return data[data.full_name == speaker]['text']

# Process the train data and create more features
def preprocessSpeakers(speakers, train_data, label_encoder):
    if (PREPROCESS_STAGE == False):
        return []

    speakers_summary = []
    for name in speakers:
        texts = getSpeakerTexts(name, train_data)
        most_common = mostCommon(texts, 25)
        speakers_summary.append({
                                'ID': label_encoder.transform([name])[0],
                                'name':name,
                                'top_N_vocab':[tup[0] for tup in most_common]
        })

    return speakers_summary

# This feature requires a prebuilt summary of the speakers (See preprocessSpeakers(...))
def speakerVocabScore(tokenized, num_speakers, vocab_lookup):    
    if (PREPROCESS_STAGE == False):
         return {}

    score_table = {}    # {speaker_id : score}

    # rally up score
    for word in tokenized:
        if word in vocab_lookup:                    # this word in the top vocab
            for speakerID in vocab_lookup[word]:    # add score to the speaker
                if speakerID in score_table:
                    score_table["SVS_" + str(speakerID)] += 1  # SVS - speaker_vocab score
                else:
                    score_table["SVS_" + str(speakerID)] = 1
    return score_table
# -------------------------------------------------------------------------
# -------------------------------------------------------------------------



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##Split training/testing

In [0]:
import numpy as np

# train/test data will only be from the sample-size
def split_train_test(data, splitpt, sample_size = 1.0):    
    length = len(data)
    rand_df = pd.DataFrame(np.random.randn(length,2))
    msk = np.random.rand(len(rand_df)) < splitpt    

    train = data[msk]
    old_len = len(train)
    train = train.sample(frac=sample_size, replace=False)    
    # print("train: old_length = ", old_len, " new_length = ", len(train))
    
    test = data[~msk]
    old_len = len(test)
    test = test.sample(frac=sample_size, replace=False)
    # print("test: old_length = ", old_len, " new_length = ", len(test))
    return train, test

In [337]:
# setup train/test dataframes - empty
train_df = pd.DataFrame(columns = df_top_N.columns)
test_df = pd.DataFrame(columns = df_top_N.columns)

# add to train/test dataframes
SAMPLE_SIZE = .25
for name in top_N_speakers:
    temp_df = df_top_N[df_top_N.full_name == name]  # only one speaker

    train, test = split_train_test(temp_df, 0.8, SAMPLE_SIZE)
    train_df = pd.concat([train_df, train])
    test_df = pd.concat([test_df, test])

print("Total points in dataset: ", len(df_top_N))
print("Sample size: ", SAMPLE_SIZE)
print("Total points in train: ", len(train_df))
print("Total points in test: ", len(test_df))
print("(For each speaker, their texts were split into 80% training and 20% testing)")
print()

Total points in dataset:  12988
Sample size:  0.25
Total points in train:  2583
Total points in test:  662
(For each speaker, their texts were split into 80% training and 20% testing)



In [0]:
labelencoder = LabelEncoder()

X_train = train_df['text']
y_train = labelencoder.fit_transform(train_df['full_name'])

X_test = test_df['text']
y_test = labelencoder.transform(test_df['full_name'])

##Pre-processing stage
Extract features from the training set

In [0]:
PREPROCESS_STAGE = False
TOP_VOCAB = {}

In [0]:
if (PREPROCESS_STAGE):
    SPEAKER_SUMMARY = preprocessSpeakers(top_N_speakers, df_top_N, labelencoder)
    num_of_speakers = len(SPEAKER_SUMMARY)

    # save time checking if a word is in a speaker's top vocab with this
    for i in range(0, num_of_speakers):
        for word in SPEAKER_SUMMARY[i]['top_N_vocab']:
            if word not in TOP_VOCAB:
                TOP_VOCAB[word] = [SPEAKER_SUMMARY[i]['ID']]
            else:
                # print(word, ":", TOP_VOCAB[word])
                TOP_VOCAB[word].append(SPEAKER_SUMMARY[i]['ID'])            


In [0]:
# sample = "And are the sole producer of 14 commodities in California. Including walnuts, which my family grows. And other products such as almonds and raisins. California's agricultural exports totaled $21 billion in 2013, representing 15% of the nation's total. What those figures show, is that farmers and ranchers are adapting."
# processText(sample)

In [341]:
### Vectorize features
print("vectorizing...", end = '')
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
X_train_vec = vectorizer.fit_transform(map(processText, list(X_train)))
X_test_vec = vectorizer.transform(map(processText, list(X_test)))
print("done", )

vectorizing...done


In [342]:

### Build model
print("training model...", end = '')
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model = model.fit(X_train_vec , y_train)

print("done", )

training model...done


In [373]:

### Accuracy Score
print("====================================")
print("Evaluation Metrics")
print("====================================")
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
predicted = model.predict(X_test_vec)
predicted_values = labelencoder.inverse_transform(predicted)
true_values = labelencoder.inverse_transform(y_test)

print("accuracy: ", accuracy_score(true_values, predicted_values))
print("precision: ")
print("   (macro): ", precision_score(true_values, predicted_values, average='macro', zero_division=0))
print("   (micro): ", precision_score(true_values, predicted_values, average='micro', zero_division=0))
print("   (weighted): ", precision_score(true_values, predicted_values, average='weighted', zero_division=0))
print("recall: ")
print("   (macro): ", recall_score(true_values, predicted_values, average='macro'))
print("   (micro): ", recall_score(true_values, predicted_values, average='micro'))
print("   (weighted): ", recall_score(true_values, predicted_values, average='weighted'))
print("f1 score:")
print("   (macro): ", f1_score(true_values, predicted_values, average='macro'))
print("   (micro): ", f1_score(true_values, predicted_values, average='micro'))
print("   (weighted): ", f1_score(true_values, predicted_values, average='weighted'))


Evaluation Metrics
accuracy:  0.29607250755287007
precision: 
   (macro):  0.18156064973715566
   (micro):  0.29607250755287007
   (weighted):  0.29692214549956963
recall: 
   (macro):  0.14359679375801662
   (micro):  0.29607250755287007
   (weighted):  0.29607250755287007
f1 score:
   (macro):  0.12392008806978896
   (micro):  0.29607250755287007
   (weighted):  0.23594762946064943
